# Revision de Croyances et Incoherence

Ce notebook couvre la revision de croyances multi-agents et les mesures d'incoherence.

**Pre-requis**: Executez d'abord [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb) pour configurer l'environnement.

**Navigation**: [Precedent](Tweety-3-Advanced-Logics.ipynb) | [Suivant](Tweety-5-Abstract-Argumentation.ipynb)


In [ ]:
# --- Initialisation JVM Tweety + Outils Externes ---
print("--- Verification JVM Tweety ---")
jvm_ready = False

import jpype
import jpype.imports
import os
import pathlib
import shutil
import platform

# === Configuration des outils externes (standalone) ===
EXTERNAL_TOOLS = {
    "CLINGO": "",
    "SPASS": "",
}

def get_tool_path(tool_name):
    """Retourne le chemin valide d'un outil ou None."""
    path_str = EXTERNAL_TOOLS.get(tool_name, "")
    if not path_str: 
        return None
    if shutil.which(path_str):
        return path_str
    path_obj = pathlib.Path(path_str)
    if path_obj.is_file():
        return str(path_obj.resolve())
    # Also check if it's a directory (for Tweety compatibility)
    if path_obj.is_dir():
        return str(path_obj.resolve())
    return None

# Auto-detection Clingo (binaire)
# IMPORTANT: TweetyProject ClingoSolver expects a DIRECTORY path, not the full executable path.
# It appends "/clingo" (Unix) or "/clingo.exe" (Windows) internally when invoking the solver.
clingo_paths = [
    shutil.which("clingo"),
    shutil.which("clingo.exe"),
    pathlib.Path("ext_tools/clingo/clingo.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/clingo/clingo.exe"),
    pathlib.Path("../ext_tools/clingo/clingo.exe"),
]
for cp in clingo_paths:
    if cp:
        if isinstance(cp, str):
            # For system PATH clingo, use the directory containing it
            cp_path = pathlib.Path(cp)
            EXTERNAL_TOOLS["CLINGO"] = str(cp_path.parent.resolve())
            break
        elif cp.exists():
            # For local clingo.exe, store the DIRECTORY path (parent)
            EXTERNAL_TOOLS["CLINGO"] = str(cp.parent.resolve())
            break

# Auto-detection SPASS
spass_paths = [
    shutil.which("SPASS"),
    shutil.which("SPASS.exe"),
    pathlib.Path("ext_tools/spass/SPASS.exe"),
    pathlib.Path("Argument_Analysis/ext_tools/spass/SPASS.exe"),
    pathlib.Path("../ext_tools/spass/SPASS.exe"),
]
for sp in spass_paths:
    if sp:
        if isinstance(sp, str):
            EXTERNAL_TOOLS["SPASS"] = sp
            break
        elif sp.exists():
            EXTERNAL_TOOLS["SPASS"] = str(sp.resolve())
            break

# === Initialisation JVM ===
if jpype.isJVMStarted():
    print("JVM deja en cours d'execution.")
    jvm_ready = True
else:
    jdk_portable = pathlib.Path("jdk-17-portable")
    if not jdk_portable.exists():
        jdk_portable = pathlib.Path("Argument_Analysis/jdk-17-portable")
    
    if jdk_portable.exists():
        zulu_dirs = list(jdk_portable.glob("zulu*"))
        if zulu_dirs:
            java_home = zulu_dirs[0]
            os.environ["JAVA_HOME"] = str(java_home.resolve())
            print(f"JDK portable trouve: {java_home.name}")
    
    if not os.environ.get("JAVA_HOME"):
        print("ERREUR: JAVA_HOME non defini et JDK portable non trouve.")
        print("JVM non disponible.")
    else:
        LIB_DIR = pathlib.Path("libs")
        if not LIB_DIR.exists():
            LIB_DIR = pathlib.Path("Argument_Analysis/libs")
        
        if LIB_DIR.exists():
            jar_files = list(LIB_DIR.glob("*.jar"))
            if jar_files:
                classpath = os.pathsep.join(str(j.resolve()) for j in jar_files)
                try:
                    jpype.startJVM(classpath=[classpath])
                    print(f"JVM demarree avec {len(jar_files)} JARs.")
                    jvm_ready = True
                except Exception as e:
                    print(f"Erreur demarrage JVM: {e}")

if jvm_ready:
    print("JVM prete pour Tweety.")
    tools = [t for t, p in EXTERNAL_TOOLS.items() if p]
    if tools:
        print(f"Outils externes: {', '.join(tools)}")

## Partie 3 : Révision de Croyances et Analyse d'Incohérence
<a id="partie3"></a>

Cette partie aborde des mécanismes de raisonnement plus avancés : comment mettre à jour des croyances face à de nouvelles informations (révision) et comment quantifier ou gérer les contradictions (incohérence). Nous nous concentrerons principalement sur la logique propositionnelle pour ces exemples.

### 3.1 Révision de Croyances Multi-Agents (CrMas)
<a id="3.1"></a>

La révision de croyances s'intéresse à l'intégration de nouvelles informations dans une base de connaissances existante, en résolvant les éventuelles contradictions de manière rationnelle (cf. postulats AGM). Tweety implémente notamment des approches pour des bases de croyances **multi-agents** où chaque information est associée à un **agent** et où un **ordre de crédibilité** peut exister entre ces agents.

* **`CrMasBeliefSet`** : Base de croyances multi-agents, nécessite un `Order<Agent>`.
* **`InformationObject`** : Encapsule une formule et l'agent source.
* **Opérateurs de Révision** (`org.tweetyproject.beliefdynamics.mas`) :
    * `CrMasRevisionWrapper`: Utilise un opérateur de révision classique (ex: Levi) sur les formules, en ignorant la structure multi-agents sauf pour prioriser la nouvelle information.
    * `CrMasSimpleRevisionOperator`: Prend en compte la crédibilité des agents de manière simple pour fusionner les informations.
    * `CrMasArgumentativeRevisionOperator`: Modélise la révision comme un processus argumentatif où les informations des agents plus crédibles peuvent "attaquer" celles des agents moins crédibles.

L'exemple suivant reprend la logique de `CrMasExample.java` et du TP C#.

In [ ]:
# --- 3.1 Révision de Croyances Multi-Agents (CrMas) ---
print("\n--- 3.1 Révision de Croyances Multi-Agents (CrMas) ---")

# Vérifier si la JVM est prête et si les imports précédents étaient OK
if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.")
# Utiliser la variable globale définie dans la cellule précédente
elif 'CrMas_Imports_OK' not in globals() or not CrMas_Imports_OK:
     print("❌ ERREUR: Imports CrMas échoués dans la cellule précédente. Impossible d'exécuter.")
else:
    print("ℹ️ JVM prête et imports CrMas OK. Exécution de l'exemple CrMas...")
    try:
        # Les imports ont déjà été tentés/réussis dans la cellule précédente.
        # On réimporte juste ce qui est nécessaire localement pour la clarté
        # Ou on utilise les références chargées (si JClass a été utilisé)

        import jpype
        from jpype.types import *

        # Références aux classes (soit importées, soit chargées via JClass)
        # Si JClass a été utilisé, ces variables existent déjà globalement
        # Si l'import direct a marché, elles sont dans le scope global
        if 'InformationObject' not in globals(): InformationObject = jpype.JClass("org.tweetyproject.beliefdynamics.InformationObject")
        if 'CrMasBeliefSet' not in globals(): CrMasBeliefSet = jpype.JClass("org.tweetyproject.beliefdynamics.mas.CrMasBeliefSet")
        if 'CrMasRevisionWrapper' not in globals(): CrMasRevisionWrapper = jpype.JClass("org.tweetyproject.beliefdynamics.mas.CrMasRevisionWrapper")
        if 'CrMasSimpleRevisionOperator' not in globals(): CrMasSimpleRevisionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.mas.CrMasSimpleRevisionOperator")
        if 'CrMasArgumentativeRevisionOperator' not in globals(): CrMasArgumentativeRevisionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.mas.CrMasArgumentativeRevisionOperator")
        if 'LeviMultipleBaseRevisionOperator' not in globals(): LeviMultipleBaseRevisionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.operators.LeviMultipleBaseRevisionOperator")
        if 'DefaultMultipleBaseExpansionOperator' not in globals(): DefaultMultipleBaseExpansionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.operators.DefaultMultipleBaseExpansionOperator")
        if 'KernelContractionOperator' not in globals(): KernelContractionOperator = jpype.JClass("org.tweetyproject.beliefdynamics.kernels.KernelContractionOperator")
        if 'RandomIncisionFunction' not in globals(): RandomIncisionFunction = jpype.JClass("org.tweetyproject.beliefdynamics.kernels.RandomIncisionFunction")
        if 'Agent' not in globals(): Agent = jpype.JClass("org.tweetyproject.agents.Agent")
        if 'DummyAgent' not in globals(): DummyAgent = jpype.JClass("org.tweetyproject.agents.DummyAgent")
        if 'Order' not in globals(): Order = jpype.JClass("org.tweetyproject.comparator.Order")

        # Imports Java util et PL (devraient être OK)
        from java.util import ArrayList, HashSet, Collection, List as JavaList
        from org.tweetyproject.logics.pl.parser import PlParser
        from org.tweetyproject.logics.pl.syntax import PlFormula, PlSignature
        from org.tweetyproject.logics.pl.reasoner import SimplePlReasoner, PlReasoner

        # Classes Java nécessaires pour les casts
        PlFormula_class = jpype.JClass("org.tweetyproject.logics.pl.syntax.PlFormula")
        Agent_class = jpype.JClass("org.tweetyproject.agents.Agent")
        PlReasoner_class = jpype.JClass("org.tweetyproject.logics.pl.reasoner.PlReasoner")
        Collection_class = jpype.JClass("java.util.Collection")


        # --- Initialisation ---
        parser = PlParser()
        agents_list = ArrayList() # Utiliser ArrayList Java
        agent1 = DummyAgent("A1"); agent2 = DummyAgent("A2"); agent3 = DummyAgent("A3")
        agents_list.add(agent1); agents_list.add(agent2); agents_list.add(agent3)

        # Ordre de crédibilité: A1 > A2 > A3
        # Order(Collection<Agent>)
        credOrder = Order(JObject(agents_list, Collection_class))
        credOrder.setOrderedBefore(agent1, agent2)
        credOrder.setOrderedBefore(agent2, agent3)
        print("\nOrdre de crédibilité:", str(credOrder))

        # Base de croyances initiale
        pl_sig_empty = PlSignature()
        # CrMasBeliefSet(Order<Agent>, Signature)
        base = CrMasBeliefSet(credOrder, pl_sig_empty)
        print("Base initiale (vide):", str(base))

        # Ajout InformationObject
        # InformationObject(Formula, Agent)
        info1_formula = parser.parseFormula("!c")
        info1_agent = agent2
        # Cast explicite pour lever toute ambiguïté potentielle
        info1 = InformationObject(JObject(info1_formula, PlFormula_class), JObject(info1_agent, Agent_class))

        info2_formula = parser.parseFormula("b")
        info2_agent = agent3
        info2 = InformationObject(JObject(info2_formula, PlFormula_class), JObject(info2_agent, Agent_class))

        info3_formula = parser.parseFormula("!b||!a")
        info3_agent = agent3
        info3 = InformationObject(JObject(info3_formula, PlFormula_class), JObject(info3_agent, Agent_class))

        base.add(info1); base.add(info2); base.add(info3)
        print("Base après ajouts:", str(base))

        # Nouvelles informations
        news_collection = HashSet() # Utiliser HashSet Java
        news1_formula = parser.parseFormula("a")
        news1_agent = agent3
        news_collection.add(InformationObject(JObject(news1_formula, PlFormula_class), JObject(news1_agent, Agent_class)))

        news2_formula = parser.parseFormula("!a||c")
        news2_agent = agent3
        news_collection.add(InformationObject(JObject(news2_formula, PlFormula_class), JObject(news2_agent, Agent_class)))


        # Affichage lisible
        news_str = ", ".join([str(info.getFormula())+" ("+str(info.getSource().getName())+")" for info in news_collection])
        print(f"\nNouvelles infos: {{ {news_str} }}")
        print(f"\nCalcul des révisions pour : {base} * {{ {news_str} }}")

        # --- Opérateurs de Révision ---

        # 1) Révision priorisée simple (Levi Wrapper)
        print("\n1. Révision Priorisée Simple (Levi Wrapper):")
        try:
            # Utiliser les classes chargées
            # KernelContractionOperator(IncisionFunction, PlReasoner)
            incision_func = RandomIncisionFunction()
            # Utiliser un SimplePlReasoner, caster en PlReasoner pour le constructeur
            pl_reasoner_instance = SimplePlReasoner()
            kernel_contract = KernelContractionOperator(incision_func, JObject(pl_reasoner_instance, PlReasoner_class))
            # LeviMultipleBaseRevisionOperator(BaseContractionOperator, BaseExpansionOperator)
            levi_operator = LeviMultipleBaseRevisionOperator(kernel_contract, DefaultMultipleBaseExpansionOperator())
            # CrMasRevisionWrapper(MultipleBaseRevisionOperator)
            revision_prio = CrMasRevisionWrapper(levi_operator)

            # revise(CrMasBeliefSet, Collection<InformationObject>)
            result_prio = revision_prio.revise(base, JObject(news_collection, Collection_class))
            print("   Résultat PRIO       :", str(result_prio))
        except jpype.JException as e_prio:
             print(f"   ❌ Erreur Java (PRIO): {e_prio.message()}")
             # print(e_prio.stacktrace()) # Pour debug
        except Exception as e_prio_py:
             print(f"   ❌ Erreur Python (PRIO): {e_prio_py}")
             import traceback; traceback.print_exc()


        # 2) Révision non priorisée simple (Crédibilité)
        print("\n2. Révision Non-Priorisée Simple (Crédibilité):")
        try:
            revision_nonprio = CrMasSimpleRevisionOperator()
            result_nonprio = revision_nonprio.revise(base, JObject(news_collection, Collection_class))
            print("   Résultat N-PRIO CRED:", str(result_nonprio))
        except jpype.JException as e_nonprio:
             print(f"   ❌ Erreur Java (N-PRIO): {e_nonprio.message()}")
             # print(e_nonprio.stacktrace())
        except Exception as e_nonprio_py:
             print(f"   ❌ Erreur Python (N-PRIO): {e_nonprio_py}")
             import traceback; traceback.print_exc()

        # 3) Révision argumentative (Crédibilité)
        print("\n3. Révision Argumentative (Crédibilité):")
        try:
            revision_arg = CrMasArgumentativeRevisionOperator()
            result_arg = revision_arg.revise(base, JObject(news_collection, Collection_class))
            print("   Résultat ARG        :", str(result_arg))
        except jpype.JException as e_arg:
             print(f"   ❌ Erreur Java (ARG): {e_arg.message()}")
             # print(e_arg.stacktrace())
        except Exception as e_arg_py:
             print(f"   ❌ Erreur Python (ARG): {e_arg_py}")
             import traceback; traceback.print_exc()


    # Gestion globale des erreurs
    except ImportError as e:
        print(f"❌ Erreur d'import initiale pour CrMas : {e}") # Ne devrait pas arriver ici
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans l'exemple CrMas: {e_java.message()}")
        print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans l'exemple CrMas: {e_gen}")
        import traceback
        traceback.print_exc()


--- 3.1 Révision de Croyances Multi-Agents (CrMas) ---
❌ ERREUR: JVM non démarrée. Impossible de continuer cet exemple.


### 3.2 Mesures d'Incohérence (PL)
<a id="3.2"></a>

Mesurer le degré d'incohérence d'une base de connaissances propositionnelle.

* **`ContensionInconsistencyMeasure`**: Basée sur le nombre minimal de variables à assigner pour trouver un modèle.
* **`MaInconsistencyMeasure` / `McscInconsistencyMeasure`**: Basées sur les MUS (Minimal Unsatisfiable Subsets). Nécessitent un énumérateur de MUS (ex: `MarcoMusEnumerator` externe ou `NaiveMusEnumerator` interne).
* **`FuzzyInconsistencyMeasure`**: Utilise une sémantique floue pour évaluer la satisfaction des formules.
* **`DSum/DMax/DHitInconsistencyMeasure`**: Basées sur la distance (ex: distance de Dalal) entre la base et les mondes possibles les plus proches la satisfaisant.

In [ ]:
# --- 3.2 Mesures d'Incohérence (PL) ---
print("\n--- 3.2 Mesures d'Incohérence (PL) ---")

if not jvm_ready:
    print("❌ ERREUR: JVM non démarrée.")
else:
    print("ℹ️ JVM prête. Exécution des exemples de mesures d'incohérence...")
    imports_inc_ok = False
    try:
        # *** AJOUT : Vérification des imports DANS cette cellule ***
        print("   Vérification imports PL de base DANS la cellule 31...")
        import jpype
        from jpype.types import *
        from org.tweetyproject.logics.pl.syntax import PlBeliefSet, PlParser, PlFormula, PlSignature, Proposition
        from org.tweetyproject.logics.pl.sat import Sat4jSolver, SatSolver
        print("   ✔️ Imports PL de base réussis DANS la cellule 31.")
        # *** FIN AJOUT ***

        # Imports spécifiques aux mesures
        from java.util import HashSet, Collection # HashSet requis pour DSum/DMax/DHit
        from org.tweetyproject.logics.pl.semantics import PossibleWorldIterator, DalalDistance # Pour DSum/DMax/DHit
        from org.tweetyproject.math.func.fuzzy import ProductNorm # Pour Fuzzy
        from org.tweetyproject.logics.pl.analysis import ContensionInconsistencyMeasure, FuzzyInconsistencyMeasure
        from org.tweetyproject.logics.pl.analysis import DSumInconsistencyMeasure, DMaxInconsistencyMeasure, DHitInconsistencyMeasure
        from org.tweetyproject.logics.commons.analysis import NaiveMusEnumerator, BeliefSetInconsistencyMeasure # Interface commune
        # Importer Ma/Mcsc spécifiquement ici
        from org.tweetyproject.logics.pl.analysis import MaInconsistencyMeasure, McscInconsistencyMeasure

        print("✔️ Imports pour Mesures d'Incohérence réussis.")
        imports_inc_ok = True

        # --- Configuration ---
        parser_inc = PlParser()
        SatSolver.setDefaultSolver(Sat4jSolver()) # Nécessaire pour certaines mesures

        # --- Exemples ---
        # (Le reste du code de la cellule 31 reste identique à ma proposition précédente)
        # ... (Collé ici pour référence, mais inchangé fonctionnellement)

        # 1. Contension (Basé sur ContensionExample.java)
        print("\n--- Mesure Contension ---")
        kb_cont = PlBeliefSet()
        formulas_cont = ["a", "!a && b", "!b", "c || a", "!c || a", "!c || d", "!d", "d", "c"]
        for f_str in formulas_cont: kb_cont.add(parser_inc.parseFormula(f_str))
        print("KB:", kb_cont)
        try:
            cont_measure = ContensionInconsistencyMeasure()
            cont_value = cont_measure.inconsistencyMeasure(kb_cont)
            print(f"Valeur Contension: {cont_value}")
        except Exception as e_cont:
            print(f"❌ Erreur Contension: {e_cont}")

        # 2. Fuzzy (Basé sur FuzzyMeasureExample.java)
        print("\n--- Mesure Fuzzy ---")
        kb_fuzzy = PlBeliefSet()
        formulas_fuzzy = ["a && !a", "!(!(a && !a))", "a && a"] # Exemple simplifié
        for f_str in formulas_fuzzy: kb_fuzzy.add(parser_inc.parseFormula(f_str))
        print("KB:", kb_fuzzy)
        try:
            # FuzzyInconsistencyMeasure(TNorm, int type) ; Type 0: SUMFUZZY_MEASURE
            fuzzy_measure = FuzzyInconsistencyMeasure(ProductNorm(), FuzzyInconsistencyMeasure.SUMFUZZY_MEASURE)
            fuzzy_value = fuzzy_measure.inconsistencyMeasure(kb_fuzzy)
            print(f"Valeur Fuzzy (ProductNorm, Sum): {fuzzy_value}")
        except Exception as e_fuzzy:
            print(f"❌ Erreur Fuzzy: {e_fuzzy}")

        # 3. DSum / DMax / DHit (Basé sur InconsistancyMeasures C#)
        print("\n--- Mesures Basées sur Distance (Dalal) ---")
        kb_dist_collection = HashSet()
        f1_dist = parser_inc.parseFormula("a && b && c")
        f2_dist = parser_inc.parseFormula("!a && !b && !c")
        kb_dist_collection.add(f1_dist)
        kb_dist_collection.add(f2_dist)
        print("Collection de formules:", kb_dist_collection)
        sig_dist = PlSignature()
        sig_dist.addAll(f1_dist.getSignature())
        sig_dist.addAll(f2_dist.getSignature())
        print("Signature:", sig_dist)
        if not sig_dist.isEmpty():
            try:
                pw_iter_dist = PossibleWorldIterator(sig_dist)
                dalal_dist = DalalDistance()
                dsum_measure = DSumInconsistencyMeasure(dalal_dist, pw_iter_dist)
                dsum_value = dsum_measure.inconsistencyMeasure(kb_dist_collection)
                print(f"Valeur DSum: {dsum_value}")
                dmax_measure = DMaxInconsistencyMeasure(dalal_dist, pw_iter_dist)
                dmax_value = dmax_measure.inconsistencyMeasure(kb_dist_collection)
                print(f"Valeur DMax: {dmax_value}")
                dhit_measure = DHitInconsistencyMeasure(dalal_dist, pw_iter_dist)
                dhit_value = dhit_measure.inconsistencyMeasure(kb_dist_collection)
                print(f"Valeur DHit: {dhit_value}")
            except jpype.JException as e_dist_java:
                 print(f"❌ Erreur Java (Mesures Distance): {e_dist_java.message()}")
            except Exception as e_dist_py:
                 print(f"❌ Erreur Python (Mesures Distance): {e_dist_py}")
        else:
            print("⚠️ Signature vide, impossible de calculer les mesures basées sur la distance.")

        # 4. Mesures Ma et Mcsc (nécessitent MUS - voir section suivante)
        print("\n--- Mesures Ma / Mcsc (nécessitent énumération MUS - voir 3.3) ---")
        kb_mus_demo = PlBeliefSet()
        formulas_mus_demo = ["a", "!a", "!a && !b", "b"]
        for f_str in formulas_mus_demo: kb_mus_demo.add(parser_inc.parseFormula(f_str))
        print("KB pour démo Ma/Mcsc:", kb_mus_demo)
        try:
            mus_enum_naive = NaiveMusEnumerator(SatSolver.getDefaultSolver())
            ma_measure_naive = MaInconsistencyMeasure(mus_enum_naive)
            ma_value = ma_measure_naive.inconsistencyMeasure(kb_mus_demo)
            print(f"Valeur Ma (Naive): {ma_value}")
            mcsc_measure_naive = McscInconsistencyMeasure(mus_enum_naive)
            mcsc_value = mcsc_measure_naive.inconsistencyMeasure(kb_mus_demo)
            print(f"Valeur Mcsc (Naive): {mcsc_value}")
        except Exception as e_ma_mcsc:
             print(f"❌ Erreur calcul Ma/Mcsc (Naive): {e_ma_mcsc}")

    # Gestion globale (inchangée)
    except ImportError as e:
        # Cette erreur sera maintenant plus spécifique si elle vient des vérifications ajoutées
        print(f"❌ Erreur d'import pour Mesures d'Incohérence : {e}")
    except jpype.JException as e_java:
        print(f"❌ Erreur Java générale dans Mesures d'Incohérence: {e_java.message()}")
        # print(e_java.stacktrace())
    except Exception as e_gen:
        print(f"❌ Erreur Python inattendue dans Mesures d'Incohérence: {e_gen}")
        import traceback; traceback.print_exc()


--- 3.2 Mesures d'Incohérence (PL) ---
❌ ERREUR: JVM non démarrée.


### 3.3 Énumération de MUS (Minimal Unsatisfiable Subsets)
<a id="3.3"></a>

Un Sous-ensemble Minimal Inconsistant (MUS) d'une base de connaissances $KB$ est un sous-ensemble $M \subseteq KB$ tel que $M$ est inconsistant, mais tout sous-ensemble propre de $M$ est consistant. Trouver les MUS est utile pour diagnostiquer les sources d'incohérence.

* **`NaiveMusEnumerator`**: Implémentation simple mais potentiellement très lente, intégrée à Tweety.
* **`MarcoMusEnumerator`**: Interface avec l'outil externe `marco.py`, beaucoup plus efficace. Nécessite d'installer MARCO et de fournir le chemin vers `marco.py`.

In [ ]:
# # --- 3.3 Énumération de MUS ---
# from org.tweetyproject.logics.pl.syntax import PlBeliefSet, PlParser
# from org.tweetyproject.logics.pl.sat import Sat4jSolver, SatSolver
# from org.tweetyproject.logics.commons.analysis import NaiveMusEnumerator
# # from org.tweetyproject.logics.pl.sat import MarcoMusEnumerator # Décommenter si Marco est configuré
# # from org.tweetyproject.logics.pl.util import CnfSampler # Pour générer des KB aléatoires
# # from org.tweetyproject.logics.pl.syntax import PropositionalSignature, Proposition # Pour CnfSampler

# # --- Configuration ---
# parser_mus = PlParser()
# SatSolver.setDefaultSolver(Sat4jSolver())

# # Exemple simple (MusExample.java simplifié)
# kb_mus_ex = PlBeliefSet()
# formulas_mus_ex = ["a", "!a", "!a && !b", "b", "c", "!c", "!a || !c"]
# for f_str in formulas_mus_ex: kb_mus_ex.add(parser_mus.parseFormula(f_str))

# print("KB pour MUS:\n", kb_mus_ex)

# # --- Énumération ---

# # Option 1: NaiveMusEnumerator
# print("\nCalcul des MUS (Naive Enumerator - peut être lent):")
# try:
#     mus_enum_naive = NaiveMusEnumerator(SatSolver.getDefaultSolver())
#     all_mus_naive = mus_enum_naive.minimalInconsistentSubsets(kb_mus_ex)
#     print(f" - Trouvé {len(all_mus_naive)} MUS:")
#     for mus in all_mus_naive:
#         print(f"   - {mus}")
# except Exception as e:
#     print(f"❌ Erreur NaiveMusEnumerator: {e}")

# # Option 2: MarcoMusEnumerator (Externe)
# # MARCO_PATH = "/path/to/your/marco.py" # <--- MODIFIEZ CECI
# # if pathlib.Path(MARCO_PATH).exists():
# #     print("\nCalcul des MUS (Marco Enumerator):")
# #     try:
# #         mus_enum_marco = MarcoMusEnumerator(MARCO_PATH)
# #         all_mus_marco = mus_enum_marco.minimalInconsistentSubsets(kb_mus_ex)
# #         print(f" - Trouvé {len(all_mus_marco)} MUS:")
# #         for mus in all_mus_marco:
# #             print(f"   - {mus}")
# #     except Exception as e:
# #         print(f"❌ Erreur MarcoMusEnumerator: {e}")
# # else:
# #     print("\nMarco MUS Enumerator non trouvé/configuré, test externe sauté.")

### 3.4 MaxSAT
<a id="3.4"></a>

MaxSAT (Maximum Satisfiability) est une généralisation du problème SAT. Étant donné un ensemble de clauses "dures" (qui doivent être satisfaites) et un ensemble de clauses "molles" (qui peuvent être violées, souvent avec un coût/poids associé), MaxSAT cherche une assignation qui satisfait toutes les clauses dures et minimise le coût total des clauses molles violées (ou maximise le poids des clauses molles satisfaites).

Tweety intègre des solveurs MaxSAT externes comme **Open-WBO**.

* **`MaxSatSolver`**: Interface abstraite.
* **`OpenWboSolver`**: Implémentation pour Open-WBO (nécessite chemin).
* **Entrée**: Une `PlBeliefSet` pour les clauses dures, et une `Map<PlFormula, Integer>` pour les clauses molles et leurs poids (coûts de violation).
* **Sortie**: Une `Interpretation` (un `PossibleWorld`) qui est une solution optimale.

In [ ]:
# # --- 3.4 MaxSAT ---
# from org.tweetyproject.logics.pl.syntax import PlBeliefSet, PlParser, PlFormula
# from org.tweetyproject.logics.pl.semantics import PossibleWorld # Interpretation est typiquement un PossibleWorld
# # from org.tweetyproject.logics.pl.sat import MaxSatSolver, OpenWboSolver # Décommenter si OpenWBO est configuré
# from java.util import HashMap

# # --- Configuration ---
# parser_maxsat = PlParser()

# # Clauses Dures (doivent être satisfaites)
# hard_clauses_bs = PlBeliefSet()
# hard_formulas = ["!a && b", "b || c", "c || d", "f || (c && g)"]
# for f_str in hard_formulas: hard_clauses_bs.add(parser_maxsat.parseFormula(f_str))

# # Clauses Molles (avec poids/coût de violation)
# # Utiliser HashMap Java: Map<PlFormula, Integer>
# soft_clauses_map = HashMap()
# soft_clauses_map.put(parser_maxsat.parseFormula("a || !b"), 25) # violer coûte 25
# soft_clauses_map.put(parser_maxsat.parseFormula("!c"), 15)      # violer coûte 15

# print("--- MaxSAT ---")
# print("Clauses Dures:", hard_clauses_bs)
# print("Clauses Molles:", soft_clauses_map) # L'affichage peut être technique

# # --- Résolution (nécessite OpenWBO) ---
# # OPENWBO_PATH = "/path/to/your/open-wbo_exec" # <--- MODIFIEZ CECI
# # if pathlib.Path(OPENWBO_PATH).exists():
# #     print("\nRésolution MaxSAT (OpenWBO):")
# #     try:
# #         maxsat_solver = OpenWboSolver(OPENWBO_PATH)
# #         # Note: MaxSatSolver.getWitness prend (BeliefSet<Hard>, Map<Soft, Weight>)
# #         witness_maxsat = maxsat_solver.getWitness(hard_clauses_bs, soft_clauses_map)

# #         if witness_maxsat:
# #             print(" - Solution (Interpretation):", witness_maxsat)
# #             # Calculer le coût de la solution
# #             # Note: MaxSatSolver.costOf n'est pas statique dans l'exemple Java, il faut une instance
# #             cost = maxsat_solver.costOf(witness_maxsat, hard_clauses_bs, soft_clauses_map)
# #             print(f" - Coût de la solution (clauses molles violées * poids): {cost}")

# #             # Vérifier si les clauses dures sont satisfaites
# #             print(f"   - Les clauses dures sont satisfaites? {witness_maxsat.satisfies(hard_clauses_bs)}")
# #         else:
# #             print(" - Aucune solution trouvée (potentiellement clauses dures inconsistantes).")

# #     except Exception as e:
# #          print(f"❌ Erreur OpenWboSolver: {e}")
# # else:
# #      print("\nOpen-WBO non trouvé/configuré, test MaxSAT externe sauté.")
# print("\n(Section MaxSAT nécessite la configuration du chemin vers OpenWBO)")

---

## Navigation

- **Precedent**: [Tweety-3-Advanced-Logics.ipynb](Tweety-3-Advanced-Logics.ipynb)
- **Suivant**: [Tweety-5-Abstract-Argumentation.ipynb](Tweety-5-Abstract-Argumentation.ipynb)
- **Index**: [Tweety-1-Setup.ipynb](Tweety-1-Setup.ipynb)
